Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A4 - Transparency
Please use the follwing structure as a starting point. Extend and change the notebook according to your needs. This structure should help you to guide you through your analysis. This notebook is the foundation for condensing your results and writing your reflection in the end. So please read what we expect from you regarding the reflection first to structure your analysis accordingly.

In [1]:
import pandas as pd
import requests
import json

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## [1] General understanding
> What is the model about and who is using it?

* What is your model about?

`reverted`: The model used for this endpoitnt predicts whether an edit will eventually be reverted.


* Why is this model useful?
 * It is useful for quality control tools
 * Helps reviewers to find potentially damaging contributions -> make the work of filtering through the Special:RecentChanges feed easier
 * Can be used for detection and removal of damaging contributions
 * There's also the need to identify good-faith contributors



* Who is using this model? 
  * **User** The model is designed to help human editors perform critical wiki-work and to increase their productivity by automating tasks like detecting vandalism and removing edits made in bad fait.
  * **Developers** The model aims to provide data for developers of tools for wikipedia (See below)
  * (**Scientists**)


* What are stakeholder or users of ORES? 
  * Volunteer tool developers and product developers at the **Wikimedia Foundation** and **Wikimedia Deutschland**
  * Authors/Editors of articles (because their edits get assessed)
  * Editors/Reviewers


* Why is this model useful to wikipedia?

It aims to improve the quality of articles and to reduce the work of reviewers/other editors.


* What applications/projects/... within wikipedia are using this model?

3rd party tools used:
  * [Edit Review Improvements (ERI)](https://www.mediawiki.org/wiki/Edit_Review_Improvements/New_filters_for_edit_review)
  * [Huggle](https://en.wikipedia.org/wiki/Wikipedia:Huggle)


## [2] API
> What does the ORES API (v3) tell you about a specific model? What functions does the API offer?

Use the API to investigate your model: https://ores.wikimedia.org/v3/#/. What do the follwing API calls do and what do they tell you about your model?

* `https://ores.wikimedia.org/v3/scores/`
* `https://ores.wikimedia.org/v3/scores/?model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki`
* `https://ores.wikimedia.org/v3/scores/enwiki?models=YOURMODELNAME&model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki?models=YOURMODELNAME&revids=SOMEIDHERE`
* `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?features=true`

> The API call `https://ores.wikimedia.org/v3/scores/` does provide information about the projects and the available models and versions for each project. 
_______________________________________________________

In [4]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/')
v3_scores = api_call.json()

In [10]:
print('There are ',len(v3_scores.keys()),' Wikipedia projects.')
v3_scores.keys()

There are  46  Wikipedia projects.


dict_keys(['arwiki', 'bnwiki', 'bswiki', 'cawiki', 'cswiki', 'dewiki', 'elwiki', 'enwiki', 'enwiktionary', 'eswiki', 'eswikibooks', 'eswikiquote', 'etwiki', 'euwiki', 'fakewiki', 'fawiki', 'fiwiki', 'frwiki', 'frwikisource', 'glwiki', 'hewiki', 'hrwiki', 'huwiki', 'idwiki', 'iswiki', 'itwiki', 'jawiki', 'kowiki', 'lvwiki', 'nlwiki', 'nowiki', 'plwiki', 'ptwiki', 'rowiki', 'ruwiki', 'simplewiki', 'sqwiki', 'srwiki', 'svwiki', 'tawiki', 'testwiki', 'trwiki', 'ukwiki', 'viwiki', 'wikidatawiki', 'zhwiki'])

In [284]:
print('ORES API (v3) provides ',len(pd.DataFrame(pd.unique(v3_scores_df['model']))),' different models')
pd.DataFrame(pd.unique(v3_scores_df['model']))

ORES API (v3) provides  10  different models


,0
0,articletopic
1,damaging
2,goodfaith
3,reverted
4,articlequality
5,draftquality
6,drafttopic
7,wp10
8,pagelevel
9,itemquality


In [6]:
accuracy_dict = {'project': [],'model':[],'version':[]}
v3_scores_df = pd.DataFrame(accuracy_dict, columns = ['model','project', 'version' ])

In [7]:
for project in v3_scores.keys():
    for model in v3_scores[project]['models'].keys():
        version = v3_scores[project]['models'][model]['version']
        v3_scores_df = v3_scores_df.append({ 'model':model, 'project': project, 'version':version}, ignore_index=True)

In [11]:
print('The reverted model is available in ',len(v3_scores_df[v3_scores_df['model'] == 'reverted']),' projects. Mainly in version 0.5.0')
v3_scores_df[v3_scores_df['model'] == 'reverted']

The reverted model is available in  10  projects. Mainly in version 0.5.0


,model,project,version
3,reverted,bnwiki,0.5.0
13,reverted,elwiki,0.5.0
21,reverted,enwiktionary,0.5.0
46,reverted,glwiki,0.5.0
49,reverted,hrwiki,0.5.0
52,reverted,idwiki,0.5.0
53,reverted,iswiki,0.5.0
91,reverted,tawiki,0.5.0
98,reverted,testwiki,0.0.3
108,reverted,viwiki,0.5.0


> The API call `https://ores.wikimedia.org/v3/scores/?model_info` provids information for all models. It provides information about the following model properties of a model: `environment`, `params`, `score_schema`, `statistics`, `type` and `version`. Furthermore with this call it is possible to see which model is available for a wikipedia project of interest. 

________________________________________________

In [14]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/?model_info')
v3_scores_model_info = api_call.json()

In [15]:
print('This API call provides information on')
print('For the wikipedia project hrwiki only the \"reverted\" model seems to be available.')
pd.DataFrame(v3_scores_model_info['hrwiki']['models'].keys())

This API call provides information on
For the wikipedia project hrwiki only the "reverted" model seems to be available.


,0
0,reverted


In [16]:
print("The modelinfo API call provides information on following properties of all models:")
print(pd.DataFrame(v3_scores_model_info['hrwiki']['models']['reverted'].keys()))

The modelinfo API call provides information on following properties of all models:
              0
0   environment
1        params
2  score_schema
3    statistics
4          type
5       version


In [18]:
def model_overview(project, model):
    env        = pd.DataFrame(v3_scores_model_info[project]['models'][model]['environment'].keys(),columns=['environment'])
    params     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['params'].keys(),columns=['params'] )
    schema     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['score_schema'].keys(), columns=['score_schema'])
    stats     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['statistics'].keys(),columns=['statistics'] )   
    model_overview = params.join([env, stats, schema], how='outer').fillna('')
    return model_overview

In [19]:
model_overview = model_overview('hrwiki', 'reverted')

In [20]:
print('Following is an overview of the information each properties provides on an model: ')
model_overview

Following is an overview of the information each properties provides on an model: 


,params,environment,statistics,score_schema
0,ccp_alpha,machine,!f1,properties
1,center,platform,!precision,title
2,criterion,processor,!recall,type
3,init,python_branch,accuracy,
4,label_weights,python_build,counts,
5,labels,python_compiler,f1,
6,learning_rate,python_implementation,filter_rate,
7,loss,python_revision,fpr,
8,max_depth,python_version,match_rate,
9,max_features,release,pr_auc,


In [22]:
print('The score schema of the model \"reverted\" in the project \"hrwiki\":')
v3_scores_model_info['hrwiki']['models']['reverted']['score_schema']

The score schema of the model "reverted" in the project "hrwiki":


{'properties': {'prediction': {'description': 'The most likely label predicted by the estimator',
   'type': 'boolean'},
  'probability': {'description': 'A mapping of probabilities onto each of the potential output labels',
   'properties': {'false': {'type': 'number'}, 'true': {'type': 'number'}},
   'type': 'object'}},
 'title': 'Scikit learn-based classifier score with probability',
 'type': 'object'}

In [23]:
stats_dict = {'metrics': [],'value':[]}
v3_stats_df = pd.DataFrame(stats_dict, columns = ['metrics','value' ])

for stat in v3_scores_model_info[project]['models'][model]['statistics'].keys():
    for label in v3_scores_model_info[project]['models'][model]['statistics'][stat].keys():
        v3_stats_df = v3_stats_df.append({ 'metrics':stat+' ('+label+')', 
                                          'value': v3_scores_model_info[project]['models'][model]['statistics'][stat][label]}, 
                                         ignore_index=True)

In [294]:
print('Here is an overview about the statistics of the model \"reverted\" for the wikipedia project \"hrwiki\":')
v3_stats_df

Here is an overview about the statistics of the model "reverted" for the wikipedia project "hrwiki":


,metrics,value
0,!f1 (labels),"{'false': 0.985, 'true': 0.375}"
1,!f1 (macro),0.68
2,!f1 (micro),0.394
3,!precision (labels),"{'false': 0.977, 'true': 0.593}"
4,!precision (macro),0.785
5,!precision (micro),0.605
6,!recall (labels),"{'false': 0.994, 'true': 0.274}"
7,!recall (macro),0.634
8,!recall (micro),0.297
9,accuracy (labels),"{'false': 0.971, 'true': 0.971}"


In [24]:
params_dict = {'param': [],'value':[]}
v3_params_df = pd.DataFrame(params_dict, columns = ['param','value' ])

for param in v3_scores_model_info[project]['models'][model]['params'].keys():
    v3_params_df = v3_params_df.append({ 'param': param, 
                                        'value': v3_scores_model_info[project]['models'][model]['params'][param]}, 
                                         ignore_index=True)

In [25]:
print('Here is an overview about the parameters of the model \"reverted\" for the wikipedia project \"hrwiki\":')
v3_params_df

Here is an overview about the parameters of the model "reverted" for the wikipedia project "hrwiki":


,param,value
0,ccp_alpha,0
1,center,1
2,criterion,friedman_mse
3,init,None
4,label_weights,{'false': 10}
5,labels,"[True, False]"
6,learning_rate,0.01
7,loss,deviance
8,max_depth,3
9,max_features,log2


In [26]:
v3_scores_model_info[project]['models'][model]['environment'].keys()

dict_keys(['machine', 'platform', 'processor', 'python_branch', 'python_build', 'python_compiler', 'python_implementation', 'python_revision', 'python_version', 'release', 'revscoring_version', 'system', 'version'])

In [27]:
environment_dict = {'environment property': [],'value':[]}
v3_environment_df = pd.DataFrame(environment_dict, columns = ['environment property','value' ])

for env_prop in v3_scores_model_info[project]['models'][model]['environment'].keys():
    v3_environment_df = v3_environment_df.append({ 'environment property': env_prop, 
                                        'value': v3_scores_model_info[project]['models'][model]['environment'][env_prop]}, 
                                         ignore_index=True)

In [29]:
print('Here is an overview about the environment properties of the model \"reverted\" for the wikipedia project \"hrwiki\":')
v3_environment_df

Here is an overview about the environment properties of the model "reverted" for the wikipedia project "hrwiki":


,environment property,value
0,machine,x86_64
1,platform,Linux-4.9.0-11-amd64-x86_64-with-debian-9.12
2,processor,
3,python_branch,
4,python_build,"[default, Sep 27 2018 17:25:39]"
5,python_compiler,GCC 6.3.0 20170516
6,python_implementation,CPython
7,python_revision,
8,python_version,3.5.3
9,release,4.9.0-11-amd64


> The API call `https://ores.wikimedia.org/v3/scores/hrwiki` provides information about available model/ version iformation for a wikipedia project of interest. 

________________________________________________

In [34]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki')
v3_scores_hrwiki = api_call.json()

In [36]:
print('For the hrwiki project only the model \"reverted\" is available in version 0.5.0.')
pd.DataFrame.from_dict(v3_scores_hrwiki['hrwiki']['models'])

For the hrwiki project only the model "reverted" is available in version 0.5.0.


,reverted
version,0.5.0


The API call `https://ores.wikimedia.org/v3/scores/enwiki?models=reverted&model_info` provides the same information as the call `https://ores.wikimedia.org/v3/scores/?model_info`. 
But with this API call the information can be requested **for the specific project and model of interest** where with the other API call information is returned for all projects and models. 

If the asked model is not available for the project then an error is returned: 

'code': 'not found',
 'message': "Models ('reverted',) not available for enwiki

________________________________________________

In [43]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki?models=reverted&model_info')
v3_scores_reverted_modelinfo = api_call.json()

In [44]:
v3_scores_reverted_modelinfo['hrwiki']['models']['reverted'].keys()

dict_keys(['environment', 'params', 'score_schema', 'statistics', 'type', 'version'])

In [45]:
stats_dict = {'metrics': [],'value':[]}
v3_stats_df = pd.DataFrame(stats_dict, columns = ['metrics','value' ])

for stat in v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'].keys():
    for label in v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'][stat].keys():
        v3_stats_df = v3_stats_df.append({ 'metrics':stat+' ('+label+')', 
                                          'value': v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'][stat][label]}, 
                                         ignore_index=True)

In [46]:
print('We checked if the stats are realy the same as the general modelinfo API call provides for the reverted model. We can confirm that it does provide consistent information.')
v3_stats_df

We checked if the stats are realy the same as the general modelinfo API call provides for the reverted model. We can confirm that it does provide consistent information.


,metrics,value
0,!f1 (labels),"{'false': 0.494, 'true': 0.919}"
1,!f1 (macro),0.707
2,!f1 (micro),0.527
3,!precision (labels),"{'false': 0.347, 'true': 0.986}"
4,!precision (macro),0.666
5,!precision (micro),0.398
6,!recall (labels),"{'false': 0.855, 'true': 0.862}"
7,!recall (macro),0.858
8,!recall (micro),0.855
9,accuracy (labels),"{'false': 0.861, 'true': 0.861}"


> The API call `https://ores.wikimedia.org/v3/scores/enwiki?models=reverted&revids=235107991` returns a prediction for a specific wikipedia article. From its output the following ifnormation can be captured: 
<br>
<br>A **prediction** stating if the revision problbly needs to be reverted or not.
<br>A **probability for false** that the revision needs to be reverted.
<br>A **probability for true** that the revision needs to be reverted. 

________________________________________________

In [47]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki?models=reverted&revids=5618117')
v3_scores_reverted_revid = api_call.json()

In [49]:
print('The score schema (from the modelinfo API call) of the model \"reverted\" in the project \"hrwiki\":')
v3_scores_model_info['hrwiki']['models']['reverted']['score_schema']

The score schema (from the modelinfo API call) of the model "reverted" in the project "hrwiki":


{'properties': {'prediction': {'description': 'The most likely label predicted by the estimator',
   'type': 'boolean'},
  'probability': {'description': 'A mapping of probabilities onto each of the potential output labels',
   'properties': {'false': {'type': 'number'}, 'true': {'type': 'number'}},
   'type': 'object'}},
 'title': 'Scikit learn-based classifier score with probability',
 'type': 'object'}

In [57]:
print('This is how a prediction of the reverted model looks like for a article revision which does not need to be reverted:')
v3_scores_reverted_revid['hrwiki']['scores']

This is how a prediction of the reverted model looks like for a article revision which does not need to be reverted:


{'5618117': {'reverted': {'score': {'prediction': False,
    'probability': {'false': 0.8265831588525849,
     'true': 0.17341684114741504}}}}}

> The API call `https://ores.wikimedia.org/v3/scores/elwiki/807457197/reverted?features=true` returns information about the models features it is looking at for making a prediction about an article revision.

In [60]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features=true')
v3_scores_reverted_features_revid = api_call.json()

In [61]:
v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted'].keys()

dict_keys(['features', 'score'])

In [62]:
feature_dict = {'feature': [],'value':[]}
v3_features_df = pd.DataFrame(feature_dict, columns = ['feature','value' ])

for feature in v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted']['features'].keys():
    v3_features_df = v3_features_df.append({ 'feature': feature, 'value':v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted']['features'][feature] }, ignore_index=True)

In [64]:
print('The reverted model has ',len(v3_features_df),' features. But most of them have the vlaue 0.')
v3_features_df.style.set_properties(**{'text-align': 'left'})

The reverted model has  78  features. But most of them have the vlaue 0.


,feature,value
0,feature.croatian.badwords.revision.diff.match_delta_decrease,0.000000
1,feature.croatian.badwords.revision.diff.match_delta_increase,0.000000
2,feature.croatian.badwords.revision.diff.match_delta_sum,0.000000
3,feature.croatian.badwords.revision.diff.match_prop_delta_decrease,0.000000
4,feature.croatian.badwords.revision.diff.match_prop_delta_increase,0.000000
5,feature.croatian.badwords.revision.diff.match_prop_delta_sum,0.000000
6,feature.croatian.informals.revision.diff.match_delta_decrease,0.000000
7,feature.croatian.informals.revision.diff.match_delta_increase,0.000000
8,feature.croatian.informals.revision.diff.match_delta_sum,0.000000
9,feature.croatian.informals.revision.diff.match_prop_delta_decrease,0.000000


In [65]:
print('Number of features where the value is different then 0: ',len(v3_features_df[v3_features_df['value'] != 0]))
v3_features_df[v3_features_df['value'] != 0]

Number of features where the value is different then 0:  29


,feature,value
24,feature.len(<datasource.tokenized(datasource.revision.parent.text)>),9.877000e+03
25,feature.len(<datasource.tokenized(datasource.revision.text)>),9.902000e+03
26,feature.len(<datasource.wikitext.revision.markups>),3.384000e+03
27,feature.len(<datasource.wikitext.revision.parent.markups>),3.380000e+03
28,feature.len(<datasource.wikitext.revision.parent.uppercase_words>),4.600000e+01
29,feature.len(<datasource.wikitext.revision.parent.words>),2.083000e+03
30,feature.len(<datasource.wikitext.revision.words>),2.089000e+03
32,feature.revision.comment.suggests_section_edit,1.000000e+00
33,feature.revision.diff.longest_new_repeated_char,1.000000e+00
34,feature.revision.diff.longest_new_token,1.000000e+00


> The API call `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?model_info` returns information about the model and the score for the spcified article revision. I think it is especialy usefull when checking some performance metrices like the *precision* or *recall* is important along with the with the *score*.  

________________________________________________

In [66]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?model_info')
v3_scores_modelinfo_revision = api_call.json()

In [67]:
print("The modelinfo API call provides information on following attributes of all models:")
print(pd.DataFrame(v3_scores_modelinfo_revision['hrwiki']['models']['reverted'].keys()))

The modelinfo API call provides information on following attributes of all models:
              0
0   environment
1        params
2  score_schema
3    statistics
4          type
5       version


In [68]:
stats_dict = {'metrics': [],'value':[]}
v3_stats_df = pd.DataFrame(stats_dict, columns = ['metrics','value' ])

for stat in v3_scores_modelinfo_revision['hrwiki']['models']['reverted']['statistics'].keys():
    for label in v3_scores_modelinfo_revision['hrwiki']['models']['reverted']['statistics'][stat].keys():
        v3_stats_df = v3_stats_df.append({ 'metrics':stat+' ('+label+')', 
                                          'value': v3_scores_modelinfo_revision['hrwiki']['models']['reverted']['statistics'][stat][label]}, 
                                         ignore_index=True)

In [69]:
v3_stats_df

,metrics,value
0,!f1 (labels),"{'false': 0.494, 'true': 0.919}"
1,!f1 (macro),0.707
2,!f1 (micro),0.527
3,!precision (labels),"{'false': 0.347, 'true': 0.986}"
4,!precision (macro),0.666
5,!precision (micro),0.398
6,!recall (labels),"{'false': 0.855, 'true': 0.862}"
7,!recall (macro),0.858
8,!recall (micro),0.855
9,accuracy (labels),"{'false': 0.861, 'true': 0.861}"


#### Conclusion:


### Feature Injection
Please check out the _feature injection_ feature of ORES: https://www.mediawiki.org/wiki/ORES/Feature_injection

**Example:**

     # Here you can get the perdiction for a revision, if the user would habe been anonymous:
     https://ores.wikimedia.org/v3/scores/enwiki/991397091/damaging?features&feature.revision.user.is_anon=true

Some of the features we think could be intresting to look at: 
    
   **User** related freatures:
    
    feature.revision.user.has_advanced_rights
    feature.revision.user.is_admin
    feature.revision.user.is_anon
    feature.revision.user.is_bot
    feature.revision.user.is_curator
    feature.revision.user.is_patroller
    feature.revision.user.is_trusted
    feature.temporal.revision.user.seconds_since_registration
    feature.enwiki.revision.cite_templates

   **Bad words** and language related:
    
    feature.croatian.badwords.revision.diff.match_delta_decrease
    feature.croatian.badwords.revision.diff.match_delta_increase
    feature.croatian.badwords.revision.diff.match_delta_sum	
    feature.croatian.badwords.revision.diff.match_prop_delta_decrease
    feature.croatian.badwords.revision.diff.match_prop_delta_increase
    feature.croatian.badwords.revision.diff.match_prop_delta_sum
    
    feature.english.badwords.revision.diff.match_delta_decrease	
    feature.english.badwords.revision.diff.match_delta_increase	
    feature.english.badwords.revision.diff.match_delta_sum	
    feature.english.badwords.revision.diff.match_prop_delta_decrease
    feature.english.badwords.revision.diff.match_prop_delta_increase
    feature.english.badwords.revision.diff.match_prop_delta_sum	


In [318]:
feature_injection_dict = {'injected feature': [], 'probability (false)':[], 'probability (true)':[],  'prediction':[]}
feature_injection_df = pd.DataFrame(feature_injection_dict, columns = ['injected feature','probability (false)','probability (true)','prediction'  ])

#### Without feature injection

In [319]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki?models=reverted&revids=5618117')
reverted_score_wo_injection = api_call.json()

In [320]:
df = reverted_score_wo_injection 
feature_injection_df = feature_injection_df.append({'injected feature': 'without feature injection', 
                                               'probability (false)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false'],  
                                               'probability (true)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['true'],
                                               'prediction':df['hrwiki']['scores']['5618117']['reverted']['score']['prediction']},
                                                ignore_index=True)

In [321]:
feature_injection_df

,injected feature,probability (false),probability (true),prediction
0,without feature injection,0.826583,0.173417,0.0


In [322]:
reverted_score_wo_injection['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false']

0.8265831588525849

#### With injecting the feature `user.is_bot=ture`

In [323]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features&feature.revision.user.is_bot=true')
reverted_score_user_bot = api_call.json()

In [324]:
df = reverted_score_user_bot
feature_injection_df = feature_injection_df.append({'injected feature': 'user.is_bot=true', 
                                               'probability (false)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false'],  
                                               'probability (true)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['true'],
                                               'prediction':df['hrwiki']['scores']['5618117']['reverted']['score']['prediction']},
                                                ignore_index=True)

In [325]:
reverted_score_user_bot['hrwiki']['scores']['5618117']['reverted']['score']

{'prediction': False,
 'probability': {'false': 0.9585143260758429, 'true': 0.04148567392415714}}

This was unexpected: The probability the treu probability for this revision to be rejected decreases if the user is a bot.

#### With injecting the feature `is_trusted=false`

In [326]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features&feature.revision.user.is_trusted=false')
reverted_score_user_trusted = api_call.json()

In [327]:
df = reverted_score_user_trusted
feature_injection_df = feature_injection_df.append({'injected feature': 'is_trusted=false', 
                                               'probability (false)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false'],  
                                               'probability (true)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['true'],
                                               'prediction':df['hrwiki']['scores']['5618117']['reverted']['score']['prediction']},
                                                ignore_index=True)

In [328]:
reverted_score_user_trusted['hrwiki']['scores']['5618117']['reverted']['score']

{'prediction': False,
 'probability': {'false': 0.8265831588525849, 'true': 0.17341684114741504}}

This feature we testd with a numeric value as the modelinfo statistics is showing a '0.0' for this freature as value. But we also tried it with a boolean. 
But we noticed that both does not show a diffrence in the outcome compared to asking without feature injection.

#### With injecting the feature `feature.croatian.badwords.revision.diff.match_delta_increase`

In [329]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features&feature.croatian.badwords.revision.diff.match_delta_increase=2')
reverted_score_cratian_bad_word_increase = api_call.json()

In [330]:
df = reverted_score_cratian_bad_word_increase
feature_injection_df = feature_injection_df.append({'injected feature': 'feature.croatian.badwords.revision.diff.match_delta_increase=2', 
                                               'probability (false)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false'],  
                                               'probability (true)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['true'],
                                               'prediction':df['hrwiki']['scores']['5618117']['reverted']['score']['prediction']},
                                                ignore_index=True)

In [331]:
reverted_score_cratian_bad_word_increase['hrwiki']['scores']['5618117']['reverted']['score']

{'prediction': False,
 'probability': {'false': 0.8125402216241415, 'true': 0.1874597783758585}}

This feature seems to be important as it also changes the probability of this revision into the negative direction.

#### With injecting the feature `feature.english.badwords.revision.diff.match_delta_increase`

In [332]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features&feature.english.badwords.revision.diff.match_delta_increase=2')
reverted_score_english_bad_word_increase = api_call.json()

In [333]:
df = reverted_score_english_bad_word_increase
feature_injection_df = feature_injection_df.append({'injected feature': 'feature.english.badwords.revision.diff.match_delta_increase=2', 
                                               'probability (false)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['false'],  
                                               'probability (true)':df['hrwiki']['scores']['5618117']['reverted']['score']['probability']['true'],
                                               'prediction':df['hrwiki']['scores']['5618117']['reverted']['score']['prediction']},
                                                ignore_index=True)

In [334]:
reverted_score_english_bad_word_increase['hrwiki']['scores']['5618117']['reverted']['score']

{'prediction': False,
 'probability': {'false': 0.8265831588525849, 'true': 0.17341684114741504}}

But an increase of bad words count in english language does not change the score. 

In [335]:
feature_injection_df

,injected feature,probability (false),probability (true),prediction
0,without feature injection,0.826583,0.173417,0.0
1,user.is_bot=ture,0.958514,0.041486,0.0
2,is_trusted=false,0.826583,0.173417,0.0
3,feature.croatian.badwords.revision.diff.match_delta_increase=2,0.812540,0.187460,0.0
4,feature.english.badwords.revision.diff.match_delta_increase=2,0.826583,0.173417,0.0


### Conclusion:

## [3] ML algorithm and training/test data
> Which machine learning model is underlying and what data is used to build the model?

* Check out `model_info` in detail.
* What does it tell you about the model performance?
* You can visualise and explain your results regarding model performance.
* What data was used to train and test the model?
* What machine learning algorithm is your model using? Please explain briefly.

* Check out `model_info` in detail.


* What does it tell you about the model performance?

`model_info` provides several measures of the model's performance.
Under the attribute `statistics` you can find the values for the confusion matrix, several related measures (recall, precision, false-positive rate) as well as AUC (Area Under Curve) for the ROC curve (Receiver Operating Characteristics).

In [102]:
v3_stats_df[v3_stats_df['metrics'].str.startswith('precision') | v3_stats_df['metrics'].str.startswith('recall') | v3_stats_df['metrics'].str.startswith('f1')]

,metrics,value
15,f1 (labels),"{'false': 0.919, 'true': 0.494}"
16,f1 (macro),0.707
17,f1 (micro),0.886
30,precision (labels),"{'false': 0.986, 'true': 0.347}"
31,precision (macro),0.666
32,precision (micro),0.935
35,recall (labels),"{'false': 0.862, 'true': 0.855}"
36,recall (macro),0.858
37,recall (micro),0.861


* Precision (TP) / (TP + FP):

Precision tells us how relevant are the positive detections. Higher the precision better is our detection mechanism. e.g. in our example precision is just 10% which is very poor. We are misclassifying a lot of healthy patients as cancer patients.


* Recall (sensitivity/ TPR) (TP)/(TP + FN):
What percentage of actual positives are detected. i.e. what percentage of real cancer patients are detected. It is computed by taking the ratio of correctly identified cancer patients (true positives) to the total number of cancer patients (true positives + false negatives).



In [100]:
predictions_dict = {'false': [15708, 2524], 'true': [ 211, 1241]}

confusion_matrix = pd.DataFrame(predictions_dict, columns = ['false','true'], index=['false','true'])

In [101]:
print('Confusion matrix: ')
confusion_matrix 

Confusion matrix: 


,false,true
false,15708,211
true,2524,1241


* What data was used to train and test the model?

The history of edits (and reverted edits) from a wiki.

* What machine learning algorithm is your model using? Please explain briefly. \
Different algorithms were used. Most wikis use GradientBoosting (bnwiki, elwiki, glwiki, hrwiki, idwiki, iswiki, tawiki, viwiki). Gradient boosting creates an ensemble learner by iteratively adding weak learners to an ensemble. Only the enwiktionary uses RandomForest and testwiki uses RevIDScorer.

## [4] Features
> Which features are used and which have the greatest influence on the prediction?

* What features is your model using?


* What do they mean?
* Which is the most important features?
* `https://ores.wikimedia.org/v3/scores/enwiki/991379667/articlequality?features=true`
* Are all models (in all languages of wikipedia), are they using the same features?

## Sample code

In [120]:


# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/YOUR-USER-NAME',
    'From': 'YOUR-EMAIL@fu-berlin.de'
}

def get_ores_data(rev_id, headers):
    
    # Define the endpoint: This is an example!
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'YOUMODELNAME',
              'revids'  : rev_id
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

***

#### Credits

We release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).